### Projects of using Spark
#### 16300200020 Yanjian Zhang

Use the linux command to delete the first 76 lines and redirect to the data_new.txt file.Then delete the last 49 lines and redirect to the data_new.txt. We can see that the total numbers of lines are 49611709

Introduce the spark package and configure the spark context.

In [1]:
import os
os.getcwd()

'/home/bigdatalab06'

In [2]:
from __future__ import print_function,division
from pyspark import SparkConf,SparkContext
from pyspark.sql import SparkSession
from itertools import islice
import re
conf = SparkConf().setAppName("turkey").setMaster("local")
sc = SparkContext(conf=conf)


In [3]:
# E1. find the eldest
data = sc.textFile('hdfs://localhost:9001/user/data_new.txt').map(lambda x : re.split("\t", x))
validYearData = data.filter(lambda x : len(x[8].split("/")[-1])==4)
male = validYearData.filter(lambda x : x[6] == 'E')
year = male.map(lambda x : (" ".join([x[2], x[3]]),int(x[8][-4:])))
name, birthyear = sorted(year.collect(), key = lambda x: x[1])[0]
print("name",name)
print("age",2019 - int(birthyear))


In [4]:
# E2. find the max character
data = sc.textFile('hdfs://localhost:9001/user/data_new.txt').map(lambda x : re.split("\t", x))
name = data.map(lambda x : "".join([x[2],x[3]]))
count = name.flatMap(lambda x: list(x)).map(lambda x: (x, 1)).reduceByKey(lambda x, y: x + y)
letter,freq = sorted(count.collect(), key = lambda x: x[1])[-1]
print("letter",letter)
print("frequency",freq)

In [5]:
# E3. the distribution of the age
def ageLevel(age):
    if age <=18:
        return '0-18'
    if age <= 28:
        return '19-28'
    if age <= 38:
        return '29-38'
    if age <= 48:
        return '39-48'
    if age <= 59:
        return '49-59'
    return '60+'
data = sc.textFile('hdfs://localhost:9001/user/data_new.txt').map(lambda x : re.split("\t", x))
validYearData = data.filter(lambda x : len(x[8].split("/")[-1])==4)  # prevent the case like \95 instead of \1995
year = validYearData.map(lambda x : (x[8][-4:]))
age = year.map(lambda x : 2019 - int(x))
level = age.map(lambda x: (ageLevel(x),1))
levelcount = level.reduceByKey(lambda x, y: x+y)
print(levelcount.collect())

In [6]:
# E4. birthmonth proportion
data = sc.textFile('hdfs://localhost:9001/user/data_new.txt').map(lambda x : re.split("\t", x))
month = data.map(lambda x : x[8].split("/")[1])
validMonth = month.filter(lambda x: int(x) in range(1,13) if x else False)
# print(month.collect()[0])

monthCount = validMonth.map(lambda x: (x,1)).reduceByKey(lambda x, y: x+y).persist()
totalCount = monthCount.map(lambda x: x[1]).reduce(lambda x,y: x+y)
monthPortion = monthCount.mapValues(lambda x: str('%.4f%%' % (x/float(totalCount)*100)))
print(monthPortion.collect())

In [7]:
# E5. sex proportion
data = sc.textFile('hdfs://localhost:9001/user/data_new.txt').map(lambda x : re.split("\t", x))
sex = data.map(lambda x : (x[6])).map(lambda x: (x,1))
sexCount = sex.reduceByKey(lambda x, y: x+y)
totalCount = sexCount.map(lambda x: x[1]).reduce(lambda x,y: x+y)
sexPortion = sexCount.mapValues(lambda x: str('%.4f%%' % (x/float(totalCount)*100)))
print("sex population",sexCount.collect())
print("sex proportion",sexPortion.collect())


In [8]:
# N1. top 10 surname for both sex
data = sc.textFile('hdfs://localhost:9001/user/data_new.txt').map(lambda x : re.split("\t", x))
surnamepair = data.map(lambda x :(x[6],x[3]))
male = surnamepair.filter(lambda x : x[0] == "E")
female = surnamepair.filter(lambda x: x[0] == "K")
maleCount = male.map(lambda x: (x[1], 1)).reduceByKey(lambda x, y: x + y)
femaleCount = female.map(lambda x: (x[1], 1)).reduceByKey(lambda x, y: x + y)
print("male top 10",maleCount.top(10, key = lambda x:x[1]))
print("female top 10",femaleCount.top(10, key = lambda x:x[1]))

In [9]:
# N2. cities' average age N3 youngest cities
import numpy as np
data = sc.textFile('hdfs://localhost:9001/user/data_new.txt').map(lambda x : re.split("\t", x))
validYearData = data.filter(lambda x : len(x[8].split("/")[-1])==4)
citypair = validYearData.map(lambda x : (x[11], 2019-int(x[8][-4:])))
cityAgeMean = citypair.groupByKey().map(lambda x: (x[0], int(np.mean(list(x[1]))))).persist()
print("cities' average age",cityAgeMean.collect())
print("youngest age cities", sorted(cityAgeMean.collect(),key = lambda x : x[1])[:5])
cityAgeMean.unpersist()


In [10]:
# N4. top 3 surname in top 10 cities
from heapq import nlargest
from operator import add
from collections import Counter
def counterTupleList(l_:list):
    dic = Counter(l_)
    keys=dic.keys()
    vals=dic.values()
    tupleList=[(key,val) for key,val in zip(keys,vals)]
    return tupleList
data = sc.textFile('hdfs://localhost:9001/user/data_new.txt').map(lambda x : re.split("\t", x))
cityNamePair = data.map(lambda x :(x[7], x[3])) 
top10cityGroup = cityNamePair.groupByKey() \
                            .sortBy(lambda x : len(x[1]),ascending=False) \
                            .zipWithIndex() \
                            .filter(lambda x: x[1] < 10) \
                            .map(lambda x : x[0])
top3surname = top10cityGroup.mapValues(lambda x : counterTupleList(x)) \
                            .mapValues(lambda x : nlargest(3, x, key = lambda t : t[1])) \
                            .mapValues(lambda x : [t[0] for t in x])
print("top 3 surname in top 10 cities", top3surname.collect())

In [11]:
# N5. top 2 month in top 10 cities
from heapq import nlargest
from collections import Counter
def counterTupleList(l_:list):
    dic = Counter(l_)
    keys=dic.keys()
    vals=dic.values()
    tupleList=[(key,val) for key,val in zip(keys,vals)]
    return tupleList
data = sc.textFile('hdfs://localhost:9001/user/data_new.txt').map(lambda x : re.split("\t", x))
cityMonthPair = data.map(lambda x :(x[7], x[8].split("/")[1]))
top10cityGroup = cityMonthPair.groupByKey() \
                            .sortBy(lambda x : len(x[1]),ascending=False) \
                            .zipWithIndex() \
                            .filter(lambda x: x[1] < 10) \
                            .map(lambda x : x[0])
top2month= top10cityGroup.mapValues(lambda x : counterTupleList(x)) \
                            .mapValues(lambda x : nlargest(2, x, key = lambda t : t[1])) \
                            .mapValues(lambda x : [t[0] for t in x])
print("top 2 month in top 10 cities", top2month.collect())

In [12]:
# N6. top 5 name for male and female
data = sc.textFile('hdfs://localhost:9001/user/data_new.txt').map(lambda x : re.split("\t", x))
namepair = data.map(lambda x :(x[6]," ".join([x[2],x[3]])))
maleNameCount = namepair.filter(lambda x : x[0] == "E").map(lambda x: (x[1],1)).reduceByKey(lambda x, y: x+y)
femaleNameCount = namepair.filter(lambda x: x[0] == "K").map(lambda x: (x[1],1)).reduceByKey(lambda x, y: x+y)
print("top 5 name of male", sorted(maleNameCount.collect(), key = lambda t: t[1], reverse = True)[:5])
print("top 5 name of female", sorted(femaleNameCount.collect(), key = lambda t: t[1], reverse = True)[:5])

top 5 name of male [('MEHMET YILMAZ', 15665), ('MUSTAFA YILMAZ', 13049), ('MEHMET KAYA', 12177), ('MEHMET DEMIR', 11947), ('AHMET YILMAZ', 10137)]
top 5 name of female [('FATMA YILMAZ', 17376), ('AYSE YILMAZ', 13475), ('EMINE YILMAZ', 11462), ('FATMA KAYA', 11424), ('FATMA DEMIR', 11207)]


In [28]:
# N7. top 3 name in top 10 cities
from heapq import nlargest
from operator import add
from collections import Counter
def counterTupleList(l_:list):
    dic = Counter(l_)
    keys=dic.keys()
    vals=dic.values()
    tupleList=[(key,val) for key,val in zip(keys,vals)]
    return tupleList
data = sc.textFile('hdfs://localhost:9001/user/data_new.txt').map(lambda x : re.split("\t", x))
cityNamePair = data.map(lambda x :(x[7], " ".join([x[2],x[3]]))) 
top10cityGroup = cityNamePair.groupByKey() \
                            .sortBy(lambda x : len(x[1]),ascending=False) \
                            .zipWithIndex() \
                            .filter(lambda x: x[1] < 10) \
                            .map(lambda x : x[0])
top3name = top10cityGroup.mapValues(lambda x : counterTupleList(x)) \
                            .mapValues(lambda x : nlargest(3, x, key = lambda t : t[1])) \
                            .mapValues(lambda x : [t[0] for t in x])
print("top 3 name in top 10 cities", top3name.collect())

top 3 name in top 10 cities [('ISTANBUL', ['MURAT YILMAZ', 'MURAT KAYA', 'MUSTAFA YILMAZ']), ('ANKARA', ['MURAT YILMAZ', 'MURAT SAHIN', 'MUSTAFA YILMAZ']), ('IZMIR', ['MUSTAFA YILMAZ', 'MURAT YILMAZ', 'MEHMET YILMAZ']), ('ADANA', ['MEHMET YILMAZ', 'MEHMET KAYA', 'MEHMET DEMIR']), ('GAZIANTEP', ['MEHMET YILMAZ', 'MEHMET KAYA', 'MEHMET YILDIRIM']), ('BURSA', ['MEHMET YILMAZ', 'AHMET YILMAZ', 'AYSE YILMAZ']), ('KAHRAMANMARAS', ['MEHMET DEMIR', 'MEHMET KOSE', 'FATMA DEMIR']), ('SANLIURFA', ['MEHMET DEMIR', 'FATMA DEMIR', 'MEHMET CIFTCI']), ('KONYA', ['MEHMET YILMAZ', 'MUSTAFA YILMAZ', 'MUSTAFA KAYA']), ('MALATYA', ['MEHMET DOGAN', 'MEHMET KAYA', 'MEHMET YILMAZ'])]


In [14]:
# from pyspark.ml.linalg import Vector,Vectors
# from pyspark.sql import Row
# from pyspark.ml import Pipeline
# from pyspark.ml.feature import IndexToString,StringIndexer,VectorIndexer
# from pyspark import SparkConf,SparkContext
# from pyspark.sql import SparkSession
# import pyspark.sql as sql

# data = sc.textFile('hdfs://localhost:9001/user/data_4000w.txt', 12).map(lambda x : re.split("\t", x))
# # classOfFirstName = data.map(lambda x: x[2]).distinct().persist()
# # print(classOfFirstName.take(100))

In [15]:
# print(len(classOfFirstName))

In [16]:
# classOfLastName = data.map(lambda x: x[3]).distinct().persist()

In [17]:
# from pyspark.sql import Row
# spark = SparkSession(sc)
# from pyspark.ml.classification import NaiveBayes
# from pyspark.ml import Pipeline
# from pyspark.ml.feature import StringIndexer
# spark.conf.set("spark.executor.memory","250g")
# spark.conf.set("spark.driver.memory","250g")

In [18]:
# def label2onehot(label,len_):
#     onehot = [0] * len_
#     onehot[int(label)] = 1
#     return onehot
    
# # currentdata = data.map(lambda x: len(x))
# # currentdata.take(10)
# currentdata = data.map(lambda x: [x[11], x[14].split()[0]])
# print(currentdata.take(10))
# # len1 = data.map(lambda x: x[2]).distinct().count() 
# # len2 = data.map(lambda x: x[3]).distinct().count()
# len1 = 627504 #  data.map(lambda x: x[2]).distinct().count() 
# len2 = 332884 #  data.map(lambda x: x[3]).distinct().count()
# print(data.map(lambda x: x[14]).distinct().count())
# print(data.map(lambda x: x[14].split()[0]).distinct().count())
# schema = ['city','street_address']
# df = spark.createDataFrame(currentdata, schema)

# indexer = [StringIndexer(inputCol = column, outputCol = column + "_index") for column in schema]

# pipeline = Pipeline(stages = indexer)
# df = pipeline.fit(df).transform(df)
# df = df.drop(*schema)

# df.show()
# # word2Vec = Word2Vec(vectorSize = 5, seed= 42,inputCol = "lastname")
# # row1 = Row("lastname")
# # embed1 = word2Vec.fit(classOfLastName.map(lambda x : [x] if x else ['0']).map(row1).toDF())

In [19]:
# classOfFirstName = data.map(lambda x: x[2]).distinct().count() 
# classOfLastName = data.map(lambda x: x[3]).distinct().count()


In [20]:
# from pyspark.mllib.regression import LabeledPoint
# from pyspark.mllib.tree import DecisionTree 
# from pyspark.mllib.tree import GradientBoostedTrees
# from pyspark.mllib.tree import RandomForest
# from pyspark.mllib.classification import NaiveBayes
# from pyspark.mllib.classification import LogisticRegressionWithSGD
# from pyspark.ml import Pipeline
# from pyspark.ml.feature import StringIndexer
# import time


# Data = df.rdd.map(lambda row: LabeledPoint(row[0], row[1:2]))
# print(Data.take(10))

# trainData, validData, testData = Data.randomSplit([0.7,0.1,0.2])

# trainData.persist()
# validData.persist()
# testData.persist()

In [21]:
# # classOfFirstName = 627504 #  data.map(lambda x: x[2]).distinct().count() 
# # classOfLastName = 332884 #  data.map(lambda x: x[3]).distinct().count()
# classOfFirstName = data.map(lambda x: x[2]).distinct().count() 
# classOfLastName = data.map(lambda x: x[3]).distinct().count()
# print(classOfFirstName, classOfLastName)

In [22]:
# def accuracy(model, validData):
#     predict = model.predict(validData.map(lambda p: p.features))
#     predict = predict.map(lambda p : float(p))
#     correct = predict.zip(validData.map(lambda x: x.label)).filter(lambda x: x[0] == x[1])
#     acc = float(correct.count())/float(predict.count())
#     return acc

# def trainEvaluateModel(trainData, validData, impurityParm, maxDepthParm, maxBinsParm, categoricalFeature):
#     print(maxBinsParm)
#     startTime = time.clock()
# #     model = DecisionTree.trainClassifier(trainData, numClasses = 2, categoricalFeaturesInfo = categoricalFeature, impurity = impurityParm, maxDepth = maxDepthParm, maxBins = maxBinsParm) 
#     model = RandomForest.trainClassifier(trainData, 2, {}, 3, seed=42)
#     acc = accuracy(model, validData)
#     duration = time.clock() - startTime
#     print("impur "+str(impurityParm)," maxDepth "+str(maxDepthParm)+" maxBins "+str(maxBinsParm)+" time "+str(duration)," accuracy "+str(acc))
#     return acc, duration, impurityParm, maxDepthParm, maxBinsParm

# impurityParm = "gini"
# def gridSearch(trainData, validData, impurityParm, maxDepthList = [9,12,24], maxBinsList = [727504,1000000,2000000]):
#     metrics = [trainEvaluateModel(trainData, validData, impurityParm, maxDepth, maxBins, dict()) for maxDepth in maxDepthList for maxBins in maxBinsList]
#     sorted_ =  sorted(metrics, key = lambda k: k[0], reverse = True)
#     best = sorted_[0]
#     print("best param, maxdepth: "+str(best[3])+" maxbins "+ str(best[4]))
#     return best

# best = gridSearch(trainData, validData, impurityParm)
# # trainEvaluateModel(trainData, testData, best[2], best[3], best[4], {})

# trainEvaluateModel(trainData, testData, best[2], best[3], best[4], {0:classOfFirstName, 1:classOfLastName})

    
    

In [23]:
# # logistic regression
# def accuracy(model, validData):
#     predict = model.predict(validData.map(lambda p: p.features))
#     predict = predict.map(lambda p : float(p))
#     correct = predict.zip(validData.map(lambda x: x.label)).filter(lambda x: x[0] == x[1])
#     acc = float(correct.count())/float(predict.count())
#     return acc

# def trainEvaluateModel(trainData, testData):
#     print(maxBinsParm)
#     startTime = time.clock()
#     model =  LogisticRegressionWithSGD.train(trainData, iterations=10)
#     acc = accuracy(model, testData)
#     duration = time.clock() - startTime
#     print(" accuracy "+str(acc))
#     return acc, duration
# trainEvaluateModel(trainData, testData)

In [24]:
# def accuracy(model, validData):
#     predict = model.predict(validData.map(lambda p: p.features))
#     predict = predict.map(lambda p : float(p))
#     correct = predict.zip(validData.map(lambda x: x.label)).filter(lambda x: x[0] == x[1])
#     acc = float(correct.count())/float(predict.count())
#     return acc

# def trainEvaluateModel(trainData, testData):
#     startTime = time.clock()
#     model =  NaiveBayes.train(trainData)
#     acc = accuracy(model, testData)
#     duration = time.clock() - startTime
#     print(" accuracy: "+str(acc)+" during "+str(duration))
#     return acc, duration, model
# acc, duration, NBmodel = trainEvaluateModel(trainData, testData)
# accuracy(NBmodel, testData)

In [25]:
# citycount = data.map(lambda x: x[11]).distinct().count()
# def accuracy(model, validData):
#     predict = model.predict(validData.map(lambda p: p.features))
#     predict = predict.map(lambda p : float(p))
#     correct = predict.zip(validData.map(lambda x: x.label)).filter(lambda x: x[0] == x[1])
#     acc = float(correct.count())/float(predict.count())
#     return acc

# def trainEvaluateModel(trainData, testData):
#     startTime = time.clock()
#     model =  RandomForest.trainClassifier(trainData,citycount, {}, 3, seed=42)
#     acc = accuracy(model, testData)
#     duration = time.clock() - startTime
#     print(" accuracy: "+str(acc)+" during "+str(duration))
#     return acc, duration, model
# acc, duration, model = trainEvaluateModel(trainData, testData)
# accuracy(model, testData)

In [26]:
# def hit_n(l_, n):
#     count = 0
#     record = {}
#     for item in l_:
#         record[item] = item
#         count += 1
#     record  = sorted(record.items() , key = lambda x:x[1], reverse = True)
#     result = []
#     for item in record:
#         result.append(item[0])
#     return result
# from pyspark.ml.classification import RandomForestClassifier
# from pyspark.ml.classification import DecisionTreeClassifier
# vectorForm = df.rdd.map(lambda row: (row[0], Vectors.dense(row[1:2])))
# Vectordf = spark.createDataFrame(vectorForm, ['label', 'features'])
# trainData, validData, testData = Vectordf.rdd.randomSplit([0.7,0.1,0.2])
# rf = RandomForestClassifier(labelCol="label", featuresCol="features", numTrees=3)
# rfmodel = rf.fit(trainData.toDF())
# predicitons = rfmodel.transform(testData.toDF())
# p1 = predictions.select('probability','label')

In [27]:
# print(out.take(1000))